In [29]:
import os
import glob
import pandas as pd
import numpy as np
from multiprocessing import Pool

In [18]:
###########################################################################################
data_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL"
result_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/query_format"
###########################################################################################
file_list = glob.glob("%s/*signif_variant_gene_pairs.txt" % data_dir)
for in_file in file_list:
    result_file = "%s/%s" % (result_dir, os.path.basename(in_file))
    df = pd.read_table(in_file, sep="\t")
#     df["chr"] = df["variant_id"].str.split("_").str[0]
#     df["start"] = df["variant_id"].str.split("_").str[1]
#     df["end"] = [int(x)+1 for x in df["start"]]
    df["search_id"] = df["variant_id"].str.split("_").str[0] + "_" + df["variant_id"].str.split("_").str[1]
#     df[["chr", "start", "end", "variant_id"]].to_csv(result_file, sep="\t", index=False)
    df[["search_id", "variant_id"]].to_csv(result_file, sep="\t", index=False)

#### To get rs id from vcf based on "search id"

In [35]:
#########################################################################################
common_vcf = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GRCh37_common_all_20180423.vcf"
data_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/query_format"
result_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/rs_id/"
########################################################################################
df = pd.read_table(common_vcf, sep="\t", header=None, comment="#")
df.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
df["search_id"] = df["CHROM"].astype(str) + "_" + df["POS"].astype(str)
df_sub = df[["search_id", "ID", "REF", "ALT"]]
df_sub.head()

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,10177,rs367896724,A,AC,.,.,RS=367896724;RSPOS=10177;dbSNPBuildID=138;SSR=...
1,1,10352,rs555500075,T,TA,.,.,RS=555500075;RSPOS=10352;dbSNPBuildID=142;SSR=...
2,1,10616,rs376342519,CCGCCGTTGCAAAGGCGCGCCG,C,.,.,RS=376342519;RSPOS=10617;dbSNPBuildID=142;SSR=...
3,1,11012,rs544419019,C,G,.,.,RS=544419019;RSPOS=11012;dbSNPBuildID=142;SSR=...
4,1,11063,rs561109771,T,G,.,.,RS=561109771;RSPOS=11063;dbSNPBuildID=142;SSR=...


In [37]:
query_list = glob.glob("%s/*.txt" % data_dir)
def get_rs_id(query):
    result_file = os.path.join(result_dir, os.path.basename(query))
    df_query = pd.read_table(query, sep="\t")
    df_result = df_query.merge(df_sub, how="left", on="search_id").dropna()
    df_result.to_csv(result_file, sep="\t", index=False)
    
pool = Pool(processes=4)
for query in query_list:
    pool.apply_async(get_rs_id, (query, ))
pool.close()
pool.join()

#### To get related gene and maf from raw file based on "variant id"
#### To get snp position from GRCh38 vcf based on "ID"

In [42]:
#####################################################################################
raw_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/"
rs_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/rs_id/"
common_vcf = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GRCh38_common_all_20180418.vcf"
result_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/GRCh38_info_dir"
#######################################################################################
df_vcf = pd.read_table(common_vcf, sep="\t", header=None, comment="#")
df_vcf.columns = ["CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]
df_vcf_sub = df_vcf[["CHROM", "POS", "ID"]]
# df.head()
file_list = glob.glob("%s/*.txt" % rs_dir)
for x in file_list:
    y = os.path.join(raw_dir, os.path.basename(x))
    df_rs = pd.read_table(x, sep="\t")
    df_raw = pd.read_table(y, sep="\t")
    df_merge = df_raw.merge(df_rs, how="left", on="variant_id").dropna()
    df_info = df_merge[["variant_id", "gene_id", "maf", "ID", "REF", "ALT"]].drop_duplicates()
    df_final = df_info.merge(df_vcf_sub, how="left", on="ID").dropna()
    result_file = os.path.join(result_dir, os.path.basename(x))
    df_final.to_csv(result_file, sep="\t", index=False)

/home/galaxy/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
# Transform txt into bed format
##########################
data_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/GRCh38_info_dir"
result_dir = "/home/galaxy/data/QTL/eQTL/sigle_tissue/GTEx_Analysis_v7_eQTL/GRCh38_bed"
###############################
file_list = glob.glob("%s/*.txt" % data_dir)
for x in file_list:
    df = pd.read_table(x, sep="\t")
    result_file = os.path.join(result_dir, os.path.basename(x).replace(".txt", ".bed"))
    df["info"] = df["gene_id"] + ";" + df["maf"].astype(str) + ";" + df["ID"] + ";" + df["REF"] + ";" + df["ALT"]
    df["chr_name"] = "chr" + df["CHROM"].astype(str)
    df["start"] = df["POS"].astype(int) - 1
    df["end"] = df["POS"].astype(int)
    df_final = df[["chr_name", "start", "end", "info"]]
#     print(df_final.head())
    df_final.to_csv(result_file, sep="\t", header=False, index=False)